In [1]:
from time import time

In [53]:
def print_stats(circuit):
    print('Depth:', circuit.depth())
    print('Num qubits:', circuit.num_qubits)
    print('Ops:', circuit.count_ops())

# UCCSD

In [54]:
from qiskit.circuit import ParameterVector
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.aqua.operators import Z2Symmetries
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator

In [55]:
def get_qubit_op(dist, molecule="LiH", basis="sto3g"):
    if molecule == "LiH":
        driver = PySCFDriver(atom="Li .0 .0 .0; H .0 .0 " + str(dist), unit=UnitsType.ANGSTROM, 
                             charge=0, spin=0, basis=basis)
    elif molecule == "BeH2":
        driver = PySCFDriver(atom="Be .0 .0 .0; H .0 .0 -" + str(dist) + "; H .0 .0 " + str(dist), 
                             unit=UnitsType.ANGSTROM, 
                             charge=0, spin=0, basis=basis)
    molecule = driver.run()
    freeze_list = [0]
    remove_list = [-3, -2]
    repulsion_energy = molecule.nuclear_repulsion_energy
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2
    remove_list = [x % molecule.num_orbitals for x in remove_list]
    freeze_list = [x % molecule.num_orbitals for x in freeze_list]
    remove_list = [x - len(freeze_list) for x in remove_list]
    remove_list += [x + molecule.num_orbitals - len(freeze_list)  for x in remove_list]
    freeze_list += [x + molecule.num_orbitals for x in freeze_list]
    ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
    ferOp, energy_shift = ferOp.fermion_mode_freezing(freeze_list)
    num_spin_orbitals -= len(freeze_list)
    num_particles -= len(freeze_list)
    ferOp = ferOp.fermion_mode_elimination(remove_list)
    num_spin_orbitals -= len(remove_list)
    qubitOp = ferOp.mapping(map_type='parity', threshold=0.00000001)
    qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles)
    shift = energy_shift + repulsion_energy
    return qubitOp, num_particles, num_spin_orbitals, shift


Try `LiH` or `BeH2` as molecules, `sto-3g` for a small basis set and `3-21g` as large basis set.
To make the circuit deeper, you can set `reps` to a value larger than 1.

In [5]:
molecule = 'LiH'
basis_set = '3-21g'
reps = 1
dist = 1.3
qubitOp, num_particles, num_spin_orbitals, shift = get_qubit_op(dist, molecule, basis_set)
var_form = UCCSD(
        num_orbitals=num_spin_orbitals,
        num_particles=num_particles,
        reps=1,
        qubit_mapping='parity'
)
t0 = time()
circuit = var_form.construct_circuit(ParameterVector('P', length=var_form.num_parameters))
print('Constructed in {}s'.format(time() - t0))

Constructed in 1.8730289936065674s


In [52]:
print_stats(circuit.decompose())

Depth: 2406
Num qubits: 10
Ops: OrderedDict([('u3', 1960), ('cx', 1312), ('cmp_dg', 14), ('cmp', 8), ('cLinRot', 8), ('cmp_dg_dg', 6), ('cLinRot_dg', 6), ('LinRot', 4), ('LinRot_dg', 3), ('h', 2), ('cu1', 1)])
Ops (decomposed): OrderedDict([('u3', 2156), ('cx', 1510), ('x', 280), ('ccx', 168), ('cry', 21), ('ry', 7), ('u1', 3), ('u2', 2)])


# QAE

In [24]:
from qiskit.aqua.algorithms import AmplitudeEstimation
from qiskit.circuit.library import EfficientSU2
from qiskit.aqua.components.uncertainty_models import NormalDistribution
from qiskit.aqua.components.uncertainty_problems import UnivariatePiecewiseLinearObjective

In [43]:
class AFactory(CircuitFactory):
    def __init__(self, objective, distribution):
        super().__init__(distribution.num_target_qubits + 1)
        self.objective = objective
        self.distribution = distribution
        self.i_state = list(range(distribution.num_target_qubits))
        self.i_target = distribution.num_target_qubits
        
    def required_ancillas(self):
        return self.objective.required_ancillas()
    
    def required_ancillas_controlled(self):
        return self.objective.required_ancillas_controlled()
        
    def build(self, qc, q, q_ancillas=None, params=None):
        qr_state = [q[i] for i in self.i_state]
        qr_target = [q[self.i_target]]
        
        self.distribution.build(qc, qr_state)
        self.objective.build(qc, q, q_ancillas=q_ancillas)
        

In [62]:
num_state_qubits = 4
breakpoints = [0, 2, 5]
slopes = [1, 2, 5]
offsets = [-1, 0, 1]
f_min = 0
f_max = 12
c_approx = 0.1
distribution = NormalDistribution(num_state_qubits)
pwl = UnivariatePiecewiseLinearObjective(num_state_qubits, 0, 10, 
                                         breakpoints, slopes, offsets,
                                         f_min, f_max, c_approx)

a_factory = AFactory(pwl, distribution)

/Users/jul/Work/Qiskit/qiskit-aqua/qiskit/aqua/components/uncertainty_problems/univariate_piecewise_linear_objective.py:164: DeprecationWarning: The qiskit.aqua.circuits.PiecewiseLinearRotation object is deprecated and will be removed no earlier than 3 months after the 0.7.0 release of Qiskit Aqua. You should use qiskit.circuit.library.PiecewiseLinearPauliRotations instead.
  # create piecewise linear Y rotation


In [63]:
m = 5
qae = AmplitudeEstimation(m, a_factory=a_factory)
t0 = time()
circuit = qae.construct_circuit()
print('Constructed in {}s'.format(time() - t0))

Constructed in 16.878931999206543s


In [64]:
print_stats(circuit)

Depth: 47750
Num qubits: 15
Ops: OrderedDict([('u1', 31139), ('cx', 27200), ('u2', 5771), ('u3', 2309), ('pw_lin', 32), ('pw_lin_dg', 31), ('iqft', 1)])
